In [ ]:
#import
import re
import math as math
import numpy as np
from collections import Counter
import json
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed


Using TensorFlow backend.


In [2]:
# setup and load input text
TEXT_LENGTH=1000000 #try less in a case of memory error
SEQ_LENGTH = 10
BATCH_SIZE = 30
LAYER_NUM = 2
HIDDEN_DIM = 500
GENERATE_LENGTH = 50
NB_EPOCH = 140 # standard:20
GENERATE_FREQ=10 #generate text every X epoch
FREQ_LIMIT=1 # words with occurency up to the value will be counted and renamed as unknown

data_dir = 'resources/LOTR.txt'
text = open(data_dir, "r").read()
text = text[0:TEXT_LENGTH] #just 1,000,000 because of memory limits

In [3]:
#convert input text word by word
wordRegEx = re.compile("(?:[a-zA-Z]+)|(?:[,;:.!'´?])")
words = wordRegEx.findall(text)

data=[x.lower() for x in words]

# have two sorted arrays - one will be reduced by FREQ_LIMIT to get final dict for generating text, 
# second one will be in origine size for translation of input text
words_to_freq=sorted(Counter(data).items(), key=lambda x: x[1], reverse=True)
words_to_freq2=sorted(Counter(data).items(), key=lambda x: x[1], reverse=True)

In [4]:
# DICTIONARY FOR TEXT GENERATION - with word unknown
sum_of_freqs = 0
x = 0
while x < len(words_to_freq):
    if words_to_freq[x][1] <= FREQ_LIMIT:
        # words_to_freq2[x][0]="unknown"
        sum_of_freqs += words_to_freq[x][1]

        del words_to_freq[x]
    x += 1

words_to_freq.append(tuple(('unknown', sum_of_freqs)))
words_to_freq = sorted(words_to_freq, key=lambda x: x[1], reverse=True)
words_sorted = [x[0] for x in words_to_freq]
gen_ix_to_word = {ix: char for ix, char in enumerate(words_sorted)}  # output dictionary for translating input text-> idx


In [5]:
# DICTIONARY WITH SUM of OCCURRENCE
textDict = []
for x in range(len(words_to_freq2)):
    if words_to_freq2[x][1] <= FREQ_LIMIT:
        textDict.append([words_to_freq2[x][0], sum_of_freqs])
    else:
        textDict.append(words_to_freq2[x])

textDict = sorted(textDict, key=lambda x: x[1], reverse=True)
ix_to_word = []
cnt = 0
for x in range(len(textDict)):  #making of indexes for word_to_number dictionary for text generation
    if textDict[x][1] != sum_of_freqs:
        cnt += 1
    ix_to_word.append([textDict[x][0], cnt])

ix_to_word = {t[0]: t[1] for t in ix_to_word}

# translated text
text_ix = [ix_to_word[value] for value in data]
print("Length of input text", len(text))
print("Number of words", len(words))
print("Unique words", len(gen_ix_to_word))
print("Deleted words", len(ix_to_word) - len(gen_ix_to_word) - 1)
print("Vocabulary size:", len(ix_to_word))


Length of input text 500000
Number of words 114065
Unique words 4854
Deleted words 1257
Vocabulary size: 6112


In [6]:
#prepare zero matrixes for each sequence
VOCAB_SIZE = int(len(gen_ix_to_word))
num_of_seq = int(math.ceil(len(text_ix) / SEQ_LENGTH))
X = np.zeros((num_of_seq, SEQ_LENGTH, VOCAB_SIZE))
y = np.zeros((num_of_seq, SEQ_LENGTH, VOCAB_SIZE))

print("for[0:" + str(math.floor(len(text_ix) / SEQ_LENGTH)) + "]")


for[0:11406]


In [7]:
#fill the matrixes
range_num = int(math.floor(len(text_ix) / SEQ_LENGTH))
for i in range(range_num):
    if i % 5000 == 0: print(i, '/', range_num)
    X_sequence_ix = text_ix[i * SEQ_LENGTH:(i + 1) * SEQ_LENGTH]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))

    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
        X[i] = input_sequence
    y_sequence_ix = text_ix[i * SEQ_LENGTH + 1:(i + 1) * SEQ_LENGTH + 1]

    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))

    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
        y[i] = target_sequence


0 / 11406
5000 / 11406
10000 / 11406


In [8]:
#Creating and compiling the Network

model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


------Creating and compiling the Network------/


In [9]:
#function for text generation
def generate_text(model, length, vocab_size, gen_ix_to_word):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    print(ix)
    y_char = [gen_ix_to_word[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    upper = True
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        word = gen_ix_to_word[ix[-1]]

        if upper:
            word = word.title()

        if word == "." or word == "?" or word == "!":
            upper = True
        else:
            upper = False

        print(word + ' ',end="",flush=True)
        ix = np.argmax(model.predict(X[:, :i + 1, :])[0], 1)
        y_char.append(word)
    return (' ').join(y_char)


In [10]:
#Generate some sample before training to know how bad it is!
generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, gen_ix_to_word)


------Generate some sample before training to know how bad it is!------
[1831]
Weeks hammered across fence easy easy fence anyway courage courage ring departure ring decide decide foolish foolish foolish gum heartening path path beneath beneath path beneath dawn dawn sawed sawed sawed fading fading fading whom never never apparently apparently apparently covered covered gown gown once once branch dies she she 

'weeks Weeks hammered across fence easy easy fence anyway courage courage ring departure ring decide decide foolish foolish foolish gum heartening path path beneath beneath path beneath dawn dawn sawed sawed sawed fading fading fading whom never never apparently apparently apparently covered covered gown gown once once branch dies she she'

In [ ]:
#Training
nb_epoch = 0

while nb_epoch < NB_EPOCH:
    print('\n\nEpoch: {}\n'.format(nb_epoch))
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, epochs=1)
    nb_epoch += 1

    # generate every GENERATE_FREQth epoch
    if nb_epoch % GENERATE_FREQ == 0 or nb_epoch == 1 or nb_epoch == 5 or nb_epoch == 10:
        gen_text = generate_text(model, GENERATE_LENGTH, VOCAB_SIZE, gen_ix_to_word)
        space = " "

        gen_text = [space, nb_epoch, space, str(gen_text)]
        text.append(gen_text)
        with open('char_{}_hidden_{}_epoch_{}.json'.format(LAYER_NUM, HIDDEN_DIM, nb_epoch), 'w') as fp:
            json.dump(text, fp)

        model.save_weights('char_{}_hidden_{}_epoch_{}.hdf5'.format(LAYER_NUM, HIDDEN_DIM, nb_epoch))




------Training if there is no trained weights specified------


Epoch: 0

Epoch 1/1
 1560/11407 [===>..........................] - ETA: 1:47 - loss: 6.4571